<a href="https://colab.research.google.com/github/couyang24/coursera/blob/master/Capstone_Project_The_Battle_of_Neighborhoods_(Week_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food Hunt in downtown Toronto (Part 2)

# Load Packages

In [13]:
!pip install pgeocode

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim

In [5]:
def locator(df, postal_code, country_code):
    import pgeocode
    import pandas

    nomi = pgeocode.Nominatim(country_code)
    df['lat'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).latitude)
    df['lng'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).longitude)

    return df

In [8]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')
wikitable = soup.find('table')
columns = [i.text.strip() for i in wikitable.find_all('th')]
data = [[j.text.strip() for j in i.find_all('td')] for i in wikitable.find_all("tr")[1:]]
trtdata = pd.DataFrame(data, columns = columns)
trtdata['Neighbourhood'] = trtdata.Neighbourhood.apply(lambda x: x.replace(' /',','))
trtdata.columns = ['PostalCode','Borough', 'Neighbourhood']

In [14]:
trtdata1 = trtdata.loc[trtdata.Borough != 'Not assigned'].reset_index(drop=True)
trtdata1.loc[trtdata1.Neighbourhood == 'Not assigned', 'Neighbourhood'] = trtdata['Borough']
trtdata2 = trtdata1.groupby(['PostalCode', 'Borough']).agg(', '.join).reset_index()
trtdata3 = locator(trtdata2, 'PostalCode', 'ca')
trtdata3 = trtdata3.dropna()

In [15]:
trtdata3.head()

,PostalCode,Borough,Neighbourhood,lat,lng
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [16]:
#@title Default title text
import numpy

In [ ]:
radius=1000
LIMIT=10
VERSION = '20200605'
lat = trtdata3.lat[0]
lng = trtdata3.lng[0]

In [ ]:
trtdata3.head()

In [ ]:
for row in range(trtdata3.shape[0]):
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        trtdata3.lat[row], 
        trtdata3.lng[row], 
        radius, 
        LIMIT)

    # make the GET request
    result = requests.get(url).json()
    
    results.append(result)

In [ ]:
clean_data = results["response"]['groups'][0]['items']

In [ ]:
clean_data[0]['venue']['name']

In [ ]:
venues_list=[]

In [ ]:
venues_list.append([(
    v['venue']['name'], 
    v['venue']['location']['lat'], 
    v['venue']['location']['lng'],  
    v['venue']['categories'][0]['name']) for v in clean_data])

In [ ]:
[item for item in venues_list[0]]

In [ ]:
venues_list